In [1]:
import pandas as pd
import glob

%matplotlib inline

from pathlib import Path
from omegaconf import OmegaConf

In [2]:
# Collect predictions

def check_num_epochs(run, num_epochs=100):
    if Path(run / "valid_log.csv").is_file():
        valid_log = pd.read_csv(run / "valid_log.csv", index_col=0)
        if len(valid_log) < num_epochs:
            print("Wrong number of epochs in run: {}".format(run))
    else:
        print("valid_log.csv not exists in run: {}".format(run))

def collect_one(model_name, run, csv_file, num_epochs=100):
    check_num_epochs(run, num_epochs=num_epochs)
    
    cfg = OmegaConf.load(run / '.hydra' / 'config.yaml')
    cfg = OmegaConf.to_container(cfg)

    run_number = cfg['data']['train']['cross_val_bucket_validation_index']
    regime, inv_temp = float(run.parent.parts[-1].rsplit('-', 1)[1]), float(run.parent.parts[-2].rsplit('-', 1)[1])

    csv_path = run / 'test_predictions' / csv_file
    if not csv_path.exists():
        print(f'Skipping not found: {csv_path}')
        return pd.DataFrame()
    
    data = pd.read_csv(csv_path)
    if data.empty:
        print(f'Pred file is empty: {csv_path}')

    data['model'] = model_name
    data['run_number'] = run_number
    data['inv_temp'] = inv_temp
    data['regime'] = regime
    
    return data

def collect_all(model_name, root, csv_file):
    root = Path(root)

    metrics = []
    for inv_temp in list(root.glob("inv_temp-*")):
        for regime in list(inv_temp.glob("regime-*")):
            for run in list(regime.glob("run-*")):
                metrics.append(collect_one(model_name, run, csv_file))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

In [3]:
# Compute metrics for each detected run

def compute_metrics(data, grouping):
    columns = ['Model', '# Run', 'Inv Temp', 'Regime', 'Dice', 'Jaccard']
    metrics = []
    
    data = data.copy().reset_index()
    grouped = data.groupby(grouping)
    
    for model_group, predictions in grouped:
        model_name, run_number, inv_temp, regime = model_group[0], model_group[1], model_group[2], model_group[3]
        
        dice = predictions['segm/dice'].values
        jaccard = predictions['segm/jaccard'].values
        
        metrics.append([model_name, run_number, inv_temp, regime, dice.mean(), jaccard.mean()])
        
    metrics_df = pd.DataFrame(metrics, columns=columns)
    
    return metrics_df

def summarize_metrics(metrics):
    mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[['Dice', 'Jaccard']].aggregate(['mean', 'std'])
    
    return mean_metrics

<h1>Evaluation</h1>

<p>Evaluate models over datasets (5-cross validation)</p>

<h2>PH2 Dataset</h2>

In [7]:
#ROOT = "./"
ROOT = "/mnt/Ext_Data/results/hebbian-medical-image-segmentation"

runs = {
    'UNet': list(Path(ROOT + '/runs/experiment=ph2/').glob('unet')),
    'H-UNet-SWTA': list(Path(ROOT + '/runs/experiment=ph2/').glob('hunet-swta')),
    'H-UNet-SWTA-FT': list(Path(ROOT + '/runs/experiment=ph2/').glob('hunet-swta_ft')),
    'H-UNet-SWTA-T': list(Path(ROOT + '/runs/experiment=ph2/').glob('hunet-swta_t')),
    'H-UNet-SWTA-T-FT': list(Path(ROOT + '/runs/experiment=ph2/').glob('hunet-swta_t_ft')),
}

In [8]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [9]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet-SWTA,0,0.001,1.0,0.347459,0.220506
1,H-UNet-SWTA,0,0.002,1.0,0.406330,0.272467
2,H-UNet-SWTA,0,0.005,1.0,0.376278,0.253494
3,H-UNet-SWTA,0,2.000,1.0,0.785376,0.659664
4,H-UNet-SWTA,0,5.000,1.0,0.794921,0.672899
5,H-UNet-SWTA,0,10.000,1.0,0.807764,0.689072
6,H-UNet-SWTA-FT,0,0.001,1.0,0.761935,0.632993
7,H-UNet-SWTA-FT,0,0.002,1.0,0.817542,0.703456
8,H-UNet-SWTA-FT,0,0.005,1.0,0.816057,0.702950
9,H-UNet-SWTA-FT,0,2.000,1.0,0.857253,0.758608


Dice       Jaccard    
                                      mean std      mean std
Model            Inv Temp Regime                            
H-UNet-SWTA      0.001    1.0     0.347459 NaN  0.220506 NaN
                 0.002    1.0     0.406330 NaN  0.272467 NaN
                 0.005    1.0     0.376278 NaN  0.253494 NaN
                 2.000    1.0     0.785376 NaN  0.659664 NaN
                 5.000    1.0     0.794921 NaN  0.672899 NaN
                 10.000   1.0     0.807764 NaN  0.689072 NaN
H-UNet-SWTA-FT   0.001    1.0     0.761935 NaN  0.632993 NaN
                 0.002    1.0     0.817542 NaN  0.703456 NaN
                 0.005    1.0     0.816057 NaN  0.702950 NaN
                 2.000    1.0     0.857253 NaN  0.758608 NaN
                 5.000    1.0     0.860508 NaN  0.766551 NaN
                 10.000   1.0     0.857023 NaN  0.763188 NaN
H-UNet-SWTA-T    0.001    1.0     0.364194 NaN  0.236520 NaN
                 0.002    1.0     0.500131 NaN  0.366627 NaN
                 0.005    1.0     0.344096 NaN  0.216778 NaN
                 2.000    1.0     0.798013 NaN  0.674931 NaN
                 5.000    1.0     0.811419 NaN  0.695058 NaN
                 10.000   1.0     0.798698 NaN  0.676844 NaN
H-UNet-SWTA-T-FT 0.001    1.0     0.781603 NaN  0.658246 NaN
                 0.002    1.0     0.793119 NaN  0.672910 NaN
                 0.005    1.0     0.779779 NaN  0.654959 NaN
                 2.000    1.0     0.851924 NaN  0.754643 NaN
                 5.000    1.0     0.866120 NaN  0.774140 NaN
                 10.000   1.0     0.853715 NaN  0.755382 NaN
UNet             1.000    1.0     0.851703 NaN  0.756285 NaN

<h2>ISIC 2016 Dataset</h2>

In [22]:
#ROOT = "./"
ROOT = "/mnt/Ext_Data/results/hebbian-medical-image-segmentation"

runs = {
    'UNet': list(Path(ROOT + '/runs/experiment=isic2016/').glob('unet')),
    'H-UNet-SWTA': list(Path(ROOT + '/runs/experiment=isic2016/').glob('hunet-swta')),
    'H-UNet-SWTA-FT': list(Path(ROOT + '/runs/experiment=isic2016/').glob('hunet-swta_ft')),
    'H-UNet-SWTA-T': list(Path(ROOT + '/runs/experiment=isic2016/').glob('hunet-swta_t')),
    'H-UNet-SWTA-T-FT': list(Path(ROOT + '/runs/experiment=isic2016/').glob('hunet-swta_t_ft')),
}

In [23]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [24]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet-SWTA,0,0.001,1.0,0.279620,0.171027
1,H-UNet-SWTA,0,0.002,1.0,0.367899,0.237425
2,H-UNet-SWTA,0,0.005,1.0,0.473021,0.334356
3,H-UNet-SWTA,0,2.000,1.0,0.737769,0.609246
4,H-UNet-SWTA,0,5.000,1.0,0.730096,0.597795
5,H-UNet-SWTA,0,10.000,1.0,0.743858,0.623220
6,H-UNet-SWTA-FT,0,0.001,1.0,0.820996,0.713543
7,H-UNet-SWTA-FT,0,0.002,1.0,0.778024,0.657678
8,H-UNet-SWTA-FT,0,0.005,1.0,0.734156,0.600639
9,H-UNet-SWTA-FT,0,2.000,1.0,0.848543,0.749035


Dice       Jaccard    
                                      mean std      mean std
Model            Inv Temp Regime                            
H-UNet-SWTA      0.001    1.0     0.279620 NaN  0.171027 NaN
                 0.002    1.0     0.367899 NaN  0.237425 NaN
                 0.005    1.0     0.473021 NaN  0.334356 NaN
                 2.000    1.0     0.737769 NaN  0.609246 NaN
                 5.000    1.0     0.730096 NaN  0.597795 NaN
                 10.000   1.0     0.743858 NaN  0.623220 NaN
H-UNet-SWTA-FT   0.001    1.0     0.820996 NaN  0.713543 NaN
                 0.002    1.0     0.778024 NaN  0.657678 NaN
                 0.005    1.0     0.734156 NaN  0.600639 NaN
                 2.000    1.0     0.848543 NaN  0.749035 NaN
                 5.000    1.0     0.846684 NaN  0.749667 NaN
                 10.000   1.0     0.837474 NaN  0.735330 NaN
H-UNet-SWTA-T    0.001    1.0     0.498905 NaN  0.355979 NaN
                 0.002    1.0     0.318617 NaN  0.199275 NaN
                 0.005    1.0     0.267750 NaN  0.163044 NaN
                 2.000    1.0     0.737928 NaN  0.607723 NaN
                 5.000    1.0     0.740001 NaN  0.611534 NaN
                 10.000   1.0     0.740710 NaN  0.616802 NaN
H-UNet-SWTA-T-FT 0.001    1.0     0.751068 NaN  0.625782 NaN
                 0.002    1.0     0.807158 NaN  0.692174 NaN
                 0.005    1.0     0.829815 NaN  0.723643 NaN
                 2.000    1.0     0.838178 NaN  0.735655 NaN
                 5.000    1.0     0.810767 NaN  0.710726 NaN
                 10.000   1.0     0.848248 NaN  0.747647 NaN
UNet             1.000    1.0     0.836816 NaN  0.735125 NaN

<h2>KvasirSEG Dataset</h2>

In [25]:
#ROOT = "./"
ROOT = "/mnt/Ext_Data/results/hebbian-medical-image-segmentation"

runs = {
    'UNet': list(Path(ROOT + '/runs/experiment=kvasirSEG/').glob('unet')),
    'H-UNet-SWTA': list(Path(ROOT + '/runs/experiment=kvasirSEG/').glob('hunet-swta')),
    'H-UNet-SWTA-FT': list(Path(ROOT + '/runs/experiment=kvasirSEG/').glob('hunet-swta_ft')),
    'H-UNet-SWTA-T': list(Path(ROOT + '/runs/experiment=kvasirSEG/').glob('hunet-swta_t')),
    'H-UNet-SWTA-T-FT': list(Path(ROOT + '/runs/experiment=kvasirSEG/').glob('hunet-swta_t_ft')),
}

In [26]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [27]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet-SWTA,0,0.001,1.0,0.261240,0.156116
1,H-UNet-SWTA,0,0.002,1.0,0.277637,0.170643
2,H-UNet-SWTA,0,0.005,1.0,0.264526,0.158610
3,H-UNet-SWTA,0,2.000,1.0,0.406727,0.272606
4,H-UNet-SWTA,0,5.000,1.0,0.409068,0.273103
5,H-UNet-SWTA,0,10.000,1.0,0.411893,0.276746
6,H-UNet-SWTA-FT,0,0.001,1.0,0.408150,0.275266
7,H-UNet-SWTA-FT,0,0.002,1.0,0.380882,0.250689
8,H-UNet-SWTA-FT,0,0.005,1.0,0.390853,0.258718
9,H-UNet-SWTA-FT,0,2.000,1.0,0.735841,0.616942


Dice       Jaccard    
                                      mean std      mean std
Model            Inv Temp Regime                            
H-UNet-SWTA      0.001    1.0     0.261240 NaN  0.156116 NaN
                 0.002    1.0     0.277637 NaN  0.170643 NaN
                 0.005    1.0     0.264526 NaN  0.158610 NaN
                 2.000    1.0     0.406727 NaN  0.272606 NaN
                 5.000    1.0     0.409068 NaN  0.273103 NaN
                 10.000   1.0     0.411893 NaN  0.276746 NaN
H-UNet-SWTA-FT   0.001    1.0     0.408150 NaN  0.275266 NaN
                 0.002    1.0     0.380882 NaN  0.250689 NaN
                 0.005    1.0     0.390853 NaN  0.258718 NaN
                 2.000    1.0     0.735841 NaN  0.616942 NaN
                 5.000    1.0     0.741005 NaN  0.620023 NaN
                 10.000   1.0     0.725275 NaN  0.604296 NaN
H-UNet-SWTA-T    0.001    1.0     0.273035 NaN  0.164780 NaN
                 0.002    1.0     0.296798 NaN  0.184530 NaN
                 0.005    1.0     0.255037 NaN  0.151997 NaN
                 2.000    1.0     0.406665 NaN  0.270880 NaN
                 5.000    1.0     0.408579 NaN  0.271835 NaN
                 10.000   1.0     0.434897 NaN  0.295786 NaN
H-UNet-SWTA-T-FT 0.001    1.0     0.384478 NaN  0.252906 NaN
                 0.002    1.0     0.400953 NaN  0.266831 NaN
                 0.005    1.0     0.393808 NaN  0.259085 NaN
                 2.000    1.0     0.701072 NaN  0.577696 NaN
                 5.000    1.0     0.741180 NaN  0.624408 NaN
                 10.000   1.0     0.745406 NaN  0.627872 NaN
UNet             1.000    1.0     0.685022 NaN  0.560719 NaN

<h2>Data Science Bowl 2018 Dataset</h2>

In [28]:
#ROOT = "./"
ROOT = "/mnt/Ext_Data/results/hebbian-medical-image-segmentation"

runs = {
    'UNet': list(Path(ROOT + '/runs/experiment=datasciencebowl2018/').glob('unet')),
    'H-UNet-SWTA': list(Path(ROOT + '/runs/experiment=datasciencebowl2018/').glob('hunet-swta')),
    'H-UNet-SWTA-FT': list(Path(ROOT + '/runs/experiment=datasciencebowl2018/').glob('hunet-swta_ft')),
    'H-UNet-SWTA-T': list(Path(ROOT + '/runs/experiment=datasciencebowl2018/').glob('hunet-swta_t')),
    'H-UNet-SWTA-T-FT': list(Path(ROOT + '/runs/experiment=datasciencebowl2018/').glob('hunet-swta_t_ft')),
}

In [29]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [30]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet-SWTA,0,0.001,1.0,0.433182,0.293950
1,H-UNet-SWTA,0,0.002,1.0,0.633607,0.479857
2,H-UNet-SWTA,0,0.005,1.0,0.169068,0.097866
3,H-UNet-SWTA,0,2.000,1.0,0.759079,0.622942
4,H-UNet-SWTA,0,5.000,1.0,0.752148,0.617175
5,H-UNet-SWTA,0,10.000,1.0,0.780533,0.653930
6,H-UNet-SWTA-FT,0,0.001,1.0,0.809131,0.692046
7,H-UNet-SWTA-FT,0,0.002,1.0,0.779566,0.657712
8,H-UNet-SWTA-FT,0,0.005,1.0,0.731529,0.598980
9,H-UNet-SWTA-FT,0,2.000,1.0,0.824140,0.713056


Dice       Jaccard    
                                      mean std      mean std
Model            Inv Temp Regime                            
H-UNet-SWTA      0.001    1.0     0.433182 NaN  0.293950 NaN
                 0.002    1.0     0.633607 NaN  0.479857 NaN
                 0.005    1.0     0.169068 NaN  0.097866 NaN
                 2.000    1.0     0.759079 NaN  0.622942 NaN
                 5.000    1.0     0.752148 NaN  0.617175 NaN
                 10.000   1.0     0.780533 NaN  0.653930 NaN
H-UNet-SWTA-FT   0.001    1.0     0.809131 NaN  0.692046 NaN
                 0.002    1.0     0.779566 NaN  0.657712 NaN
                 0.005    1.0     0.731529 NaN  0.598980 NaN
                 2.000    1.0     0.824140 NaN  0.713056 NaN
                 5.000    1.0     0.833603 NaN  0.728412 NaN
                 10.000   1.0     0.832328 NaN  0.729832 NaN
H-UNet-SWTA-T    0.001    1.0     0.606491 NaN  0.466019 NaN
                 0.002    1.0     0.656834 NaN  0.515459 NaN
                 0.005    1.0     0.636921 NaN  0.492823 NaN
                 2.000    1.0     0.748851 NaN  0.611852 NaN
                 5.000    1.0     0.752805 NaN  0.616452 NaN
                 10.000   1.0     0.731121 NaN  0.592810 NaN
H-UNet-SWTA-T-FT 0.001    1.0     0.805338 NaN  0.684946 NaN
                 0.002    1.0     0.787010 NaN  0.661773 NaN
                 0.005    1.0     0.823677 NaN  0.710350 NaN
                 2.000    1.0     0.832033 NaN  0.723323 NaN
                 5.000    1.0     0.801898 NaN  0.686990 NaN
                 10.000   1.0     0.828268 NaN  0.717549 NaN
UNet             1.000    1.0     0.837379 NaN  0.729981 NaN

<h2><s>BrainMRI Dataset</s></h2>

In [5]:
ROOT = "./"

runs = {
    #'UNet': list(Path(ROOT + '/runs/experiment=brainMRI/').glob('unet')),
    'H-UNet-SWTA': list(Path(ROOT + '/runs/experiment=brainMRI/').glob('hunet-swta')),
    'H-UNet-SWTA-FT': list(Path(ROOT + '/runs/experiment=brainMRI/').glob('hunet-swta_ft')),
    'H-UNet-SWTA-T': list(Path(ROOT + '/runs/experiment=brainMRI/').glob('hunet-swta_t')),
    'H-UNet-SWTA-T-FT': list(Path(ROOT + '/runs/experiment=brainMRI/').glob('hunet-swta_t_ft')),
}

In [6]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [7]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet-SWTA,0,0.001,1.0,0.111565,0.055784
1,H-UNet-SWTA,0,0.002,1.0,0.046428,0.023943
2,H-UNet-SWTA,0,0.005,1.0,0.042155,0.022117
3,H-UNet-SWTA,0,2.000,1.0,0.290193,0.175805
4,H-UNet-SWTA,0,5.000,1.0,0.344112,0.224951
5,H-UNet-SWTA,0,10.000,1.0,0.410864,0.262340
6,H-UNet-SWTA-FT,0,0.001,1.0,0.494738,0.296692
7,H-UNet-SWTA-FT,0,0.002,1.0,0.505196,0.314623
8,H-UNet-SWTA-FT,0,0.005,1.0,0.414225,0.269342
9,H-UNet-SWTA-FT,0,2.000,1.0,0.779051,0.488952


Dice       Jaccard    
                                      mean std      mean std
Model            Inv Temp Regime                            
H-UNet-SWTA      0.001    1.0     0.111565 NaN  0.055784 NaN
                 0.002    1.0     0.046428 NaN  0.023943 NaN
                 0.005    1.0     0.042155 NaN  0.022117 NaN
                 2.000    1.0     0.290193 NaN  0.175805 NaN
                 5.000    1.0     0.344112 NaN  0.224951 NaN
                 10.000   1.0     0.410864 NaN  0.262340 NaN
H-UNet-SWTA-FT   0.001    1.0     0.494738 NaN  0.296692 NaN
                 0.002    1.0     0.505196 NaN  0.314623 NaN
                 0.005    1.0     0.414225 NaN  0.269342 NaN
                 2.000    1.0     0.779051 NaN  0.488952 NaN
                 5.000    1.0     0.682396 NaN  0.426571 NaN
                 10.000   1.0     0.800233 NaN  0.494404 NaN
H-UNet-SWTA-T    0.001    1.0     0.036169 NaN  0.019438 NaN
                 0.002    1.0     0.033649 NaN  0.017958 NaN
                 0.005    1.0     0.041833 NaN  0.022122 NaN
                 2.000    1.0     0.141114 NaN  0.085040 NaN
                 5.000    1.0     0.312814 NaN  0.188067 NaN
                 10.000   1.0     0.332276 NaN  0.208705 NaN
H-UNet-SWTA-T-FT 0.001    1.0     0.692508 NaN  0.421780 NaN
                 0.002    1.0     0.596641 NaN  0.379436 NaN
                 0.005    1.0     0.392437 NaN  0.243738 NaN
                 2.000    1.0     0.780675 NaN  0.492736 NaN
                 5.000    1.0     0.766337 NaN  0.475115 NaN
                 10.000   1.0     0.809096 NaN  0.496670 NaN